In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim

from utils import *
from models import *

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
d = 1
m = 8
t = 200
b = 1000
lamda = 0.2
distance = 0.1
qpsk = False
coherent = False

array = ULA(m, lamda)
array.build_sensor_positions(distance)
array.build_array_manifold()

angle_max = torch.pi/2-0.1
angle_min = -torch.pi/2 + 0.1

theta = torch.rand(1) * (angle_max - angle_min) + angle_min
A = array.get_steering_vector(theta)

In [ ]:
SNRs = [0, 5, 10, 15, 20]
methods = ["MUSIC", 
           "Root-MUSIC", 
           "DA-MUSIC", 
           "DA-MUSIC v2", 
           "RNN", 
           "CLRB"]
values = {
    "MUSIC": [],
    "Root-MUSIC": [],
    "DA-MUSIC": [],
    "DA-MUSIC v2": [],
    "RNN": [],
    "CLRB": []
}

if qpsk:
    path = 'saved_models_qpsk/'
else:
    path = 'saved_models_rmspe/'

for snr in SNRs:

    if qpsk:
        x = torch.zeros(b, d, t, dtype=torch.complex64)
        for i in range(b):
            x[i] = generate_qpsk_symbols(d, t)
    else:
        x = (torch.randn(b, d, t) + 1j * torch.randn(b, d, t)) / sqrt(2) 

    n = (torch.randn(b, m, t) + 1j * torch.randn(b, m, t)) / sqrt(2) / 10 ** (snr / 20)
    y = A @ x + n
    y = y.transpose(1, 2)

    da_music = DA_MUSIC(m, d, array, dev)
    da_music.load_state_dict(torch.load(path+'da_music_'+str(snr)+'dB.pth', weights_only=True))
    values['DA-MUSIC'].append(torch.sqrt(torch.mean(((da_music(y.to(dev)).cpu() - theta + torch.pi/2) % torch.pi - torch.pi/2) ** 2)).detach().cpu())

    da_music_v2 = DA_MUSIC_v2(m, d, array, dev)
    da_music_v2.load_state_dict(torch.load(path+'da_music_v2_'+str(snr)+'dB.pth', weights_only=True))
    values['DA-MUSIC v2'].append(torch.sqrt(torch.mean(((da_music_v2(y.to(dev)).cpu() - theta + torch.pi/2) % torch.pi - torch.pi/2) ** 2)).detach().cpu())

    rnn = RNN(m, d, dev)
    rnn.load_state_dict(torch.load(path+'rnn_'+str(snr)+'dB.pth', weights_only=True))
    values['RNN'].append(torch.sqrt(torch.mean(((rnn(y.to(dev)).cpu() - theta + torch.pi/2) % torch.pi - torch.pi/2) ** 2)).detach().cpu())

    results_music = torch.zeros(b, d)
    for i in range(b):
        results_music[i], _, _ = MUSIC(y[i].T, d, array)
    values['MUSIC'].append(torch.sqrt(torch.mean((results_music - theta) ** 2)))

    result_root_music = Root_MUSIC(y, d, array)
    values['Root-MUSIC'].append(torch.sqrt(torch.mean((result_root_music - theta) ** 2)))

    sources_cov = torch.eye(d, dtype=torch.complex64)
    noise_sigma2 = 1 / 10 ** (snr/10)
    crb = CRLB_stochastic(array, theta, sources_cov, noise_sigma2, t)
    values['CLRB'].append(torch.sqrt(crb).squeeze().float().cpu().detach().numpy())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot
plt.figure(figsize=(10, 6))
for method in methods:
    plt.plot(SNRs, values[method], label=method)

# Customizing the plot
plt.yscale("log")
plt.title("Performance Comparison Across SNR Levels", fontsize=14)
plt.xlabel("SNR (dB)", fontsize=12)
plt.ylabel("RMSPE (rad)", fontsize=12)
plt.xticks(SNRs)
plt.grid(True, which="both", linewidth=0.5)
plt.legend(title="Methods")
plt.tight_layout()

# Show plot
plt.show()